In [43]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from keras.preprocessing.text import Tokenizer
from keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from keras.models import Model
import matplotlib.pyplot as plt

In [44]:
#importing the dataset
with open('replies2.json') as content:
  data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
# print(data)

In [45]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from keras_preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [46]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
# print("output length: ",output_length)

6
number of unique words :  117


In [47]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
4/4 [==============================] - 1s 5ms/step - loss: 3.2962 - accuracy: 0.0194
Epoch 2/200
4/4 [==============================] - 0s 5ms/step - loss: 3.2927 - accuracy: 0.0874
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 3.2902 - accuracy: 0.1359
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 3.2873 - accuracy: 0.1359
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 3.2847 - accuracy: 0.1359
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 3.2815 - accuracy: 0.1359
Epoch 7/200
4/4 [==============================] - 0s 4ms/step - loss: 3.2785 - accuracy: 0.1359
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 3.2745 - accuracy: 0.1359
Epoch 9/200
4/4 [==============================] - 0s 3ms/step - loss: 3.2709 - accuracy: 0.1359
Epoch 10/200
4/4 [==============================] - 0s 4ms/step - loss: 3.2661 - accuracy: 0.1068
Epoch 11/200
4/4 [===========

In [48]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Bot-404 : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

1/1 [==============================] - 0s 270ms/step
Bot-404 :  Hi there, how can I help?
1/1 [==============================] - 0s 16ms/step
Bot-404 :  Good to see you again
1/1 [==============================] - 0s 107ms/step
Bot-404 :  Hello
1/1 [==============================] - 0s 32ms/step
Bot-404 :  See you!
